In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
df = pd.read_csv("bloomberg_data.csv", encoding = 'utf8', index_col=False, parse_dates=['date'], date_parser=dateparse)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102183 entries, 0 to 102182
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  102183 non-null  int64         
 1   ticker      102183 non-null  object        
 2   date        102183 non-null  datetime64[ns]
 3   PX_HIGH     38572 non-null   float64       
 4   PX_LOW      38571 non-null   float64       
 5   PX_LAST     102182 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 4.7+ MB
None


In [2]:
import os
import sys
from datetime import datetime
import pandas as pd

testdir = os.getcwd() + '/'
if testdir != "":
    testdir = testdir + '/'
sys.path.append(testdir + "..")
sys.path.append(os.path.dirname(os.path.abspath('')))

import dxapi
#source = 'dxtick://ec2-13-53-158-252.eu-north-1.compute.amazonaws.com:54011'
#dest = 'dxtick://timebase:8011'
# Timebase URL specification, pattern is "dxtick://<host>:<port>"
timebase = 'dxtick://timebase:8011'
import dxapi 
try:
    # Create timebase connection
    print("Step 1")
    db = dxapi.TickDb.createFromUrl(timebase)
    
    # Open in read-write mode
    db.open(False)
    
    print('Connected to ' + timebase) 
    
    # Define name of the stream    
    streamKey = 'Demo_Saeed_1'
    
    # Get stream from the timebase
    stream = db.getStream(streamKey)
    
    # Create a Message Loader for the selected stream and provide loading options
    loader = stream.createLoader(dxapi.LoadingOptions())
    
    # Create Bar message
    barMessage = dxapi.InstrumentMessage()
    
    # Define message type name according to the Timebase schema type name
    # For the polymorphic streams, each message should have defined typeName to distinct messages on Timebase Server level.
    barMessage.typeName = 'MyBarMessage'
   
    print('Start loading to ' + streamKey)   
    
    # get current time in UTC
    now = datetime.utcnow() - datetime(1970, 1, 1)
    
    # Define message timestamp as Epoch time in nanoseconds 
    ns = now.total_seconds() * 1e9 + now.microseconds * 1000;
    
    barMessage.instrumentType = 'EQUITY'
    
  
    for row in df.iterrows():
        
         # get current time in UTC
        now = row[1]['date'] - datetime(1970, 1, 1)
    
        # Define message timestamp as Epoch time in nanoseconds 
        ns = now.total_seconds() * 1e9 + now.microseconds * 1000;
        
        barMessage.symbol = row[1]['ticker']   
 
        barMessage.PX_LAST = row[1]['PX_LAST']
       
    
        # Send message
        loader.send(barMessage)     
    # close Message Loader
    loader.close()
    loader = None
    
finally:
    # database connection should be closed anyway
    if db.isOpen():
        db.close()
        print("Connection " + timebase + " closed.")

Step 1
Connected to dxtick://timebase:8011
Connection dxtick://timebase:8011 closed.


AttributeError: 'NoneType' object has no attribute 'createLoader'